In [1]:
!pip install numpy


In [2]:
!pip install librosa


In [3]:
pip install audiomentations

In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from audiomentations import Compose, AddGaussianNoise, PitchShift, TimeStretch



In [ ]:
def extract_features(audio_file, max_length=100):
   
    y, sr = librosa.load(audio_file, sr=None)
   
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
  
    if mfccs.shape[1] < max_length:
        mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :max_length]
  
    flat_mfccs = np.ravel(mfccs)
    return flat_mfccs


In [ ]:
def load_audio_data(folder):
    features = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".wav"):
            audio_file = os.path.join(folder, filename)
            feature = extract_features(audio_file)
            features.append(feature)
           
            labels.append(0 if folder.endswith("synthesized") else 1)
    return np.array(features), np.array(labels)

In [ ]:
synthesized_features, synthesized_labels = load_audio_data("Synthesized")
real_features, real_labels = load_audio_data("Real")

In [ ]:
X = np.concatenate([synthesized_features, real_features], axis=0)
y = np.concatenate([synthesized_labels, real_labels], axis=0)

In [ ]:
print(X)
print(y)

[[-5.8804547e+02 -5.8678967e+02 -5.8221613e+02 ...  4.9131055e+00
   9.4823112e+00  1.0015978e+01]
 [-5.5057635e+02 -5.4940955e+02 -5.4948175e+02 ...  2.4698269e+00
   3.7205911e+00  7.9475927e+00]
 [-6.7503906e+02 -6.7334485e+02 -6.7465015e+02 ... -2.9918668e+00
  -3.7620754e+00 -4.7810335e+00]
 ...
 [-5.3539752e+02 -4.5006326e+02 -4.2542838e+02 ... -2.3322124e+01
  -1.8485149e+01 -1.5440374e+01]
 [-5.4485077e+02 -5.4485077e+02 -5.3745990e+02 ... -2.3748636e-01
   1.0813623e+01  1.1556265e+01]
 [-6.1951062e+02 -5.7963068e+02 -5.3632379e+02 ... -3.1138439e+01
  -2.7247402e+01 -9.1156502e+00]]
[1 1 1 ... 1 1 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#X_train_augmented = np.array([augment(samples=x, sample_rate=44100) for x in X_train])


In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.7))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))

Train on 4200 samples, validate on 1800 samples
Epoch 1/2
  32/4200 [..............................] - ETA: 1s - loss: 0.0000e+00 - acc: 1.0000

4200/4200 [==============================] - 1s 160us/sample - loss: 0.1072 - acc: 0.9993 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/2
4200/4200 [==============================] - 1s 158us/sample - loss: 0.2147 - acc: 0.9995 - val_loss: 0.0000e+00 - val_acc: 1.0000


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 1.0
